## **PARSING IBM VACANCY**

In [1]:
############################################
###     Research Trending Vacancies      ###
###     Sber Dep. Research&Innovation    ### 
###   Ivanov Arseny, Sergey Bratchikov   ###
###       A. Efimov, D. Asonov           ###
############################################

In [2]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [3]:
from collections import Counter

In [4]:
fake = faker.Faker(locale='en')

In [5]:
ibm_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'origin': 'https://www.ibm.com',
    'referer': 'https://www.ibm.com/'
}

In [6]:
API_URL = "http://jobsapi-internal.m-cloud.io/api/stjobbulk"

In [7]:
search_payload = {
    'organization': 2242,
    'limitkey': '4A8B5EF8-AA98-4A8B-907D-C21723FE4C6B',
    'facet': 'publish_to_cws:true',
    'fields': 'title,id,update_date,primary_country,description,primary_category,level,url,brand'
}

In [8]:
result = requests.get(API_URL, headers=ibm_headers, params=search_payload)
result

<Response [200]>

In [9]:
result.json()['totalHits'], len(result.json()['queryResult'])

(13203, 13203)

In [10]:
result.json()['queryResult'][0].keys()

dict_keys(['id', 'title', 'primary_country', 'description', 'primary_category', 'level', 'brand', 'url', 'update_date'])

In [11]:
Counter([j['primary_category'] for j in result.json()['queryResult']]).most_common(10)

[('Technical Specialist', 4874),
 ('Consultant', 2428),
 ('Software Development & Support', 1479),
 ('Finance', 1165),
 ('Sales', 814),
 ('Architect', 616),
 ('Project Management', 458),
 ('Human Resources', 249),
 ('Enterprise Operations', 214),
 ('Other', 186)]

In [12]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('•', '\n')).strip()

In [18]:
about_pattern = re.compile(r"Introduction(.+?)Your Role and Responsibilities", flags=re.DOTALL|re.IGNORECASE)
responsibilities_pattern = re.compile(r"Your Role and Responsibilities(.+?)Required Technical and Professional Expertise", flags=re.DOTALL|re.IGNORECASE)
qualifications_pattern = re.compile(r"Required Technical and Professional Expertise(.+?)Preferred Technical and Professional Expertise", flags=re.DOTALL|re.IGNORECASE)

In [20]:
job_dicts = []
for job_info in tqdm(result.json()['queryResult']):
    full_description = clear_string(job_info['description'])

    about = re.search(about_pattern, full_description)
    if about:
        about = about.group(1)

    responsibilities = re.search(responsibilities_pattern, full_description)
    if responsibilities:
        responsibilities = responsibilities.group(1)

    qualifications = re.search(qualifications_pattern, full_description)
    if qualifications:
        qualifications = qualifications.group(1)

    if not qualifications and not responsibilities and not about:
        print(f"Skipping vacancy {job_info['url']}")

    job_dict = {
        'title': job_info['title'],
        'internal_id' : job_info['id'],
        'url': job_info['url'],
        'description': about,
        'responsibilities': responsibilities,
        'qualifications': qualifications,
        'company': 'IBM',
        'grade': job_info['level'],
        'category': job_info['primary_category'],
        'publish_date': parser.parse(job_info['update_date'])
    }
    job_dicts.append(job_dict)
len(job_dicts)

  4%|▍         | 541/13203 [00:00<00:05, 2429.18it/s]

Skipping vacancy https://careers.ibm.com/job/14855197/bpo-project-leader-pe-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/14922666/-project-manager-architect-it-specialist-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/14971229/-ibm-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/14214108/-foc-research-tokyo-student-intern-intern-trl-ibm-japan-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15072653/fullstack-java-per-lima-pe/
Skipping vacancy https://careers.ibm.com/job/14680472/-ibm-consulting-nagoya-jp/
Skipping vacancy https://careers.ibm.com/job/15132490/-data-ai-expert-lab-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15017893/-pm-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15037807/l-der-de-integraci-n-bogota-co/
Skipping vacancy https://careers.ibm.com/job/15232237/-ibm-pm-it-it-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15232232/-ibm-pm-it-it-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15232235/-ibm-pm-i

  9%|▉         | 1209/13203 [00:00<00:05, 2005.42it/s]

Skipping vacancy https://careers.ibm.com/job/15500549/-ibm-pm-it-it-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15558790/-cx-digital-commerce-cx-pm-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15558791/-ibm-consulting-garage-team-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15837611/-ibm-dx-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/14789157/-ibm-it-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15232233/-ibm-pm-it-it-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15364384/-ibm-pm-it-it-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15364383/-ibm-pm-it-it-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/15818913/security-consultant-iam-management-consultant-cloud-specialist-remote/
Skipping vacancy https://careers.ibm.com/job/14976119/consultant-technique-m3-infor-h-f-remote-ibm-cic-remote/
Skipping vacancy https://careers.ibm.com/job/15055991/d-veloppeur-exp-riment-business-intelligence-h-f-remote-ibm-cic-remote/

 12%|█▏        | 1613/13203 [00:00<00:05, 1987.53it/s]

...

Skipping vacancy https://careers.ibm.com/job/15636352/desenvolvedor-a-de-aplicac-es-fullstack-remote/
Skipping vacancy https://careers.ibm.com/job/16147182/desarrollador-de-aplicaciones-remote/
Skipping vacancy https://careers.ibm.com/job/16155768/consultora-o-sap-basis-remote/
Skipping vacancy https://careers.ibm.com/job/16155769/consultor-a-sap-fi-remote/
Skipping vacancy https://careers.ibm.com/job/16158644/est-gio-em-opera-es-de-vendas-hortol-ndia-br/
Skipping vacancy https://careers.ibm.com/job/16158645/l-der-t-cnico-remote/
Skipping vacancy https://careers.ibm.com/job/16160923/devops-specialist-hortol-ndia-br/
Skipping vacancy https://careers.ibm.com/job/14598354/-ibm-ibm-partner-success-engineer-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/14598355/-ibm-ibm-partner-success-engineer-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/13641669/-ibm-security-soc-shift-tokyo-jp/
Skipping vacancy https://careers.ibm.com/job/13677107/-ibm-tokyo-jp/
Skipping vacancy https:

 93%|█████████▎| 12226/13203 [00:06<00:00, 1768.27it/s]

Skipping vacancy https://careers.ibm.com/job/14782037/associ-adjoint-services-dint-gration-mulesoft-ottawa-ca/
Skipping vacancy https://careers.ibm.com/job/15181751/associ-adjoint-secteur-nergie-et-services-publics-montreal-ca/
Skipping vacancy https://careers.ibm.com/job/15185195/professionnel-technique-de-marque-aiops-observabilit-et-gestion-de-markham-ca/
Skipping vacancy https://careers.ibm.com/job/15405025/repr-sentant-des-pr-ventes-de-stockage-flashsystem-montreal-ca/
Skipping vacancy https://careers.ibm.com/job/15425388/partenaire-associ-donn-es-et-ia-montreal-ca/
Skipping vacancy https://careers.ibm.com/job/15447968/analyste-de-la-conformit-informatique-ottawa-ca/
Skipping vacancy https://careers.ibm.com/job/15480626/ing-nieur-en-automatisation-d-quipement-bromont-ca/
Skipping vacancy https://careers.ibm.com/job/15552904/d-veloppeur-g-n-raliste-dapplications-infonuagiques-azure-ottawa-ca/
Skipping vacancy https://careers.ibm.com/job/15552905/microsoft-dynamics-365-finance-opera

100%|██████████| 13203/13203 [00:06<00:00, 2023.71it/s]


13203

In [21]:
snapshot = pd.DataFrame(job_dicts)
print(len(snapshot))
snapshot.sample(5)

13203


,title,internal_id,url,description,responsibilities,qualifications,company,grade,category,publish_date
876,Industry Consultant: Industry.Banking,15685672,https://careers.ibm.com/job/15685672/industry-...,IBM’s Industry Consultants are essential to o...,"As Business Analyst, you are required to work...",Minimum 6+ years of experience in a Business ...,IBM,Professional,Consultant,2022-05-19 02:12:35+00:00
1396,[ISE] Cloud Engineer,15914900,https://careers.ibm.com/job/15914900/-ise-clou...,"At IBM, work is more than a job - it's a call...",Refer to Japanese Job Description,Refer to Japanese Job Description. For applic...,IBM,Professional,Technical Specialist,2022-05-30 11:05:08+00:00
11437,"Technical Support Manager, Software - REMOTE",14371257,https://careers.ibm.com/job/14371257/technical...,"At IBM, work is more than a job - it's a call...",Provide technical leadership pre-sales in the...,4 years of Engineering with Deep OSS and SW s...,IBM,Professional,Software Development & Support,2022-07-03 02:39:30+00:00
2036,Application Developer: Adobe Experience Manager,15804024,https://careers.ibm.com/job/15804024/applicati...,"As an Application Developer, you will lead IB...","You will translate complex functional, techni...",Minimum 4+ years in development using DevOps ...,IBM,Professional,Technical Specialist,2022-06-10 15:31:47+00:00
2326,Technical Consultant: zOS,15840575,https://careers.ibm.com/job/15840575/technical...,"At IBM, work is more than a job - it's a call...",Managing and developing Mainframe Application...,2-4 Years of Mainframe Developer Experience. ...,IBM,Professional,Consultant,2022-06-08 23:14:54+00:00


In [22]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13203 entries, 0 to 13202
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype                  
---  ------            --------------  -----                  
 0   title             13203 non-null  object                 
 1   internal_id       13203 non-null  int64                  
 2   url               13203 non-null  object                 
 3   description       12513 non-null  object                 
 4   responsibilities  12513 non-null  object                 
 5   qualifications    12513 non-null  object                 
 6   company           13203 non-null  object                 
 7   grade             13203 non-null  object                 
 8   category          13203 non-null  object                 
 9   publish_date      13203 non-null  datetime64[ns, tzutc()]
dtypes: datetime64[ns, tzutc()](1), int64(1), object(8)
memory usage: 1.0+ MB


In [23]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'03-07-2022'

In [24]:
snapshot.to_csv(f'../data/ibm/{current_date}.csv')
snapshot.to_csv(f'../data/ibm/{current_date}.tsv', sep='\t')

#### Проверка на единичной вакансии

In [125]:
full_description = clear_string(result.json()['queryResult'][0]['description'])
full_description

"IntroductionAt IBM, work is more than a job - it's a calling: To build. To design. To code. To consult. To think along with clients and sell. To make markets. To invent. To collaborate. Not just to do something better, but to attempt things you've never thought possible. Are you ready to lead in this new era of technology and solve some of the world's most challenging problems? If so, lets talk.Your Role and ResponsibilitiesAbout the RoleThe IBM Security Product Management team is seeking an experienced Product Manager who is technical, collaborative, and truly excited about building great endpoint security products. In this role, you will bring in-depth knowledge of the endpoint and security analytics market to lead the evolution of IBM Security’s visibility, detection, and prevention technologies for QRadar XDR portfolio. You should be able to translate a big picture vision into an execution strategy backed with market validation and customer insights. You will work closely with oth

In [126]:
re.search(about_pattern, full_description).group(1)

"At IBM, work is more than a job - it's a calling: To build. To design. To code. To consult. To think along with clients and sell. To make markets. To invent. To collaborate. Not just to do something better, but to attempt things you've never thought possible. Are you ready to lead in this new era of technology and solve some of the world's most challenging problems? If so, lets talk."

In [127]:
re.search(responsibilities_pattern, full_description).group(1)

'About the RoleThe IBM Security Product Management team is seeking an experienced Product Manager who is technical, collaborative, and truly excited about building great endpoint security products. In this role, you will bring in-depth knowledge of the endpoint and security analytics market to lead the evolution of IBM Security’s visibility, detection, and prevention technologies for QRadar XDR portfolio. You should be able to translate a big picture vision into an execution strategy backed with market validation and customer insights. You will work closely with other product managers, engineering, research, product marketing, sales, service and support. The successful candidate will have the ability to influence cross-functional teams in the company.Responsibilities\n Act as the product leader for initiatives that enhance Endpoint Detection and Response (EDR) visibility, detection, and prevention for Windows, Linux and macOS.\n Utilize strategic insight and organizational skills to id

In [128]:
re.search(qualifications_pattern, full_description).group(1)

'Demonstrated experience of product management in EDR and/or AV; previous work in malware and attack analysis, research, investigation, and response highly desirable\n Hands-on threat or investigation analyst experience in a SOC/SOAR highly desirable\n Curious about new technologies, systems, and tools\n Excellent communication skills, both verbal and written, with the ability to properly translate and articulate positioning and technology\n Demonstrated ability to collaborate with peers in research, engineering, and product marketing\n Ability to prioritize numerous simultaneous tasks\n Proven ability to work effectively with both local and remote teams\n This position requires up to 25% travel to customer and IBM locations worldwide'